# Степенной метод

In [365]:
import numpy as np

In [366]:
def power_iteration(A: np.matrix, num_iterations: int) -> np.matrix:
    # Ideally choose a random vector
    # To decrease the chance that our vector
    # Is orthogonal to the eigenvector
    b_k = np.random.rand(A.shape[1], 1)
    # b_k = np.matrix("1;"
    #                 "1")

    # print(A, "\n", b_k)

    for _ in range(num_iterations):
        # calculate the matrix-by-vector product Ab
        b_k1 = np.dot(A, b_k)

        # calculate the norm
        b_k1_norm = np.linalg.norm(b_k1)

        # re normalize the vector
        b_k = b_k1 / b_k1_norm

    return b_k

In [367]:
def calc(A: np.matrix, b: np.matrix) -> np.matrix:
    return float((b.T @ A @ b) 
                 / 
                 (b.T @ b))

In [368]:
A: np.matrix = np.matrix("2 1 0;"
              "0 2 1;"
              "0 0 2")
r_k1 = power_iteration(A, 100000)

In [369]:
mu_k1: float = float((r_k1.T @ A @ r_k1) / (r_k1.T @ r_k1))
mu_k1

C:\Users\artem\AppData\Local\Temp\ipykernel_21132\1627451438.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mu_k1: float = float((r_k1.T @ A @ r_k1) / (r_k1.T @ r_k1))


2.000039999910826

In [370]:
A: np.matrix = np.matrix("1 2;"
                         "1 0")
r_k1 = power_iteration(A, 100000)
mu_k1: float = calc(A, r_k1)
mu_k1

C:\Users\artem\AppData\Local\Temp\ipykernel_21132\2676492484.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float((b.T @ A @ b)


2.0

# Обратный степенной

In [371]:
def inverse_power_iteration(A: np.matrix, num_iterations: int) -> np.matrix:
    b_k = np.random.rand(A.shape[1], 1)
    # b_k = np.matrix()
    print(b_k, "\n")

    # print(A, "\n", b_k)
    E = np.eye(A.shape[0], dtype=int)
    mu = 0
    for _ in range(num_iterations):

        # print(A - mu * E)
        inv = np.invert(A - mu * E)

        b_top = inv @ b_k
        # print(b_top)
        c_k = np.linalg.norm(b_top)
        # print(b_top, c_k, b_top / c_k)
        # print(b_top / c_k)

        b_k = b_top / c_k

    print("\n", b_k)
    return b_k

In [372]:
A: np.matrix = np.matrix("1 3 -2 0;"
                         "1 1 4 7;"
                         "4 7 11 23;"
                         "52 66 2 0")  # -0.65
A, calc(A, power_iteration(A, 1000)), calc(A, inverse_power_iteration(A, 10))

[[0.69546866]
 [0.64058003]
 [0.99080701]
 [0.7398758 ]] 


 [[0.02808188]
 [0.26239086]
 [0.73851521]
 [0.62044962]]


C:\Users\artem\AppData\Local\Temp\ipykernel_21132\2676492484.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float((b.T @ A @ b)


(matrix([[ 1,  3, -2,  0],
         [ 1,  1,  4,  7],
         [ 4,  7, 11, 23],
         [52, 66,  2,  0]]),
 29.37066149896965,
 32.51734303252207)

In [382]:
A: np.matrix = np.matrix("1 -2 -1;"
                         "-1 1 1;"
                         "1 0 -1")  # 2 - 0
A, calc(A, power_iteration(A, 100000000)), calc(A, inverse_power_iteration(A, 100000000))

C:\Users\artem\AppData\Local\Temp\ipykernel_21132\2676492484.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float((b.T @ A @ b)


[[0.60750885]
 [0.93989404]
 [0.91524412]] 


 [[-0.62952747]
 [ 0.77694098]
 [ 0.00760798]]


(matrix([[ 1, -2, -1],
         [-1,  1,  1],
         [ 1,  0, -1]]),
 2.0000000000000004,
 2.4731122500437137)